In [1]:
import sxobsplan

from pathlib import Path
import pandas as pd
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import EarthLocation

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import make_axes_locatable
import _rcparams
import requests, re

In [2]:
obsdate = "2025-11-10" # %Y-%m-%d
obsname = "lsgt" # observatory name

In [4]:
def query_visible_target(base_url, obs_constraints, sb_constraints):
    
    '''
    
    Get JPL/api data from given url and constraints.
    
    '''

    params   = {**obs_constraints, **sb_constraints} # combine all constraints into a single dictionary
    
    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:  # normal successful results
            posts = response.json()
            return posts
        else:
            print('Error:', response.status_code)
            return None
    except requests.exceptions.RequestException as e:
        print('Error:', e)
        return None

In [5]:
# observatory coordinates
dict_observatory = {
    "lsgt"     : ("Q69", 149.0644, -31.2733, 1.165, 15.7*u.arcmin), # Siding Spring Observatory
    "lemmon"   : (None, -110.7893, 32.4420, 2.791, 28.1*u.arcmin) # Lemmon Mt. Observatory
} # mpc-code, lat (deg), long (deg), alt (km), fov

# base URL for the observability API
base_url         = 'https://ssd-api.jpl.nasa.gov/sbwobs.api'      

# orbit class
sb_class_type    = ['CEN', 'JFc', 'ETc', 'CTc', 'PAR', 'HYP', 'COM', 'HTC', 'JFC']
'''
CEN: Objects with orbits between Jupiter and Neptune (5.5 au < a < 30.1 au)
JFC: Jupiter-family comet, classical definition (P < 20 y)
JFc: Jupiter-family comet, as defined by Levison and Duncan (2 < Tj < 3)
ETc: Encke-type comet, as defined by Levison and Duncan (Tj > 3; a < aJ)
CTc: Chiron-type comet, as defined by Levison and Duncan (Tj > 3; a > aJ)
HTC: Halley-type comet, classical definition (20 y < P < 200 y)
PAR: Comets on parabolic orbits (e = 1.0)
HYP: Comets on hyperbolic orbits (e > 1.0)
COM: Comet orbit not matching any defined orbit class
TNO: Objects with orbits outside Neptune (a > 30.1 au)
APO: Near-Earth asteroid orbits which cross the Earth’s orbit similar to that of 1862 Apollo (a > 1.0 au; q < 1.017 au)
MBA: Asteroids with orbital elements constrained by (2.0 au < a < 3.2 au; q > 1.666 au)
'''

# observation constraints
# https://ssd-api.jpl.nasa.gov/doc/sbwobs.html
obs_constraints = {
    'helio-max'  : 10    , # maximum heliocentric distance                               [au]
    'vmag-max'   : 18    , # V-magnitude limit                                           [mag]
    'elev-min'   : 30    , # minimum elevation                                           [deg]
    'time-min'   : 60    , # minimum exposure time                                       [min]
    # 'elong-min'  : 0     , # minimum solar elongation                                    [deg]
    # 'elong-max'  : 180   , # maximum solar elongation                                    [deg]
    'glat-min'   : 20    , # minimum angular distance from the galactiv plane            [deg]
    'output-sort': 'vmag', # sort records according to this variable.
    'optical'    : True  , # observe at night
    'fmt-ra-dec' : False   # RA & Dec format (True: sexagesimal, False: decimal degrees)
}

# small-body filter constraints
# https://ssd-api.jpl.nasa.gov/doc/sbdb_filter.html
sb_constraints = {
#    'sb-kind'  : 'c',           # asteroid-only (a) or comet-only (c)
#    'sb-class' : 'JFC',         # orbital class (CEN, JFC, HTC, PAR, HYP, COM)
    'sb-xfrag' : True           # exclude all comet fragments
}

In [6]:
obs_constraints["obs-time"] = obsdate
obs_constraints["mpc-code"] = dict_observatory[obsname][0] # LSGT (Siding Spring Observatory)
obs_constraints

{'helio-max': 10,
 'vmag-max': 18,
 'elev-min': 30,
 'time-min': 60,
 'glat-min': 20,
 'output-sort': 'vmag',
 'optical': True,
 'fmt-ra-dec': False,
 'obs-time': '2025-11-10',
 'mpc-code': 'Q69'}

In [7]:
# Query visibilty
visible_target = pd.DataFrame() 

for sb_class in sb_class_type:
    
    sb_constraints["sb-class"] = sb_class
    
        # retrieve data from Horizons
    horizons_observable = query_visible_target(base_url, obs_constraints, sb_constraints)

    if horizons_observable is None or 'data' not in horizons_observable: # if failed to retrieve data.
        print(f"No data found for date: {obsdate} / class_type: {sb_class}")
        continue

    df                = pd.DataFrame(horizons_observable['data'], columns=horizons_observable['fields'])
    df['date_str']    = obsdate
    df['observatory'] = obsname
    df['sb_class']    = sb_class
    
    visible_target = pd.concat([visible_target, df], ignore_index=True)

In [8]:
# string to numeric types
for column in visible_target.columns:
    try:
        visible_target[column] = pd.to_numeric(visible_target[column])
    except (ValueError, TypeError):
        pass

# Vmag into float type
visible_target['fVmag'] = visible_target['Vmag'].apply(lambda x: float(re.findall(r'[\d.]+', x)[0]))

# convert 'Max. time observable (hh:mm)' to total minutes
try:
    visible_target['Max. time observable'] = visible_target['Max. time observable'].apply(
        lambda t: int(t.split(':')[0]) * 60 + int(t.split(':')[1])
    )
except AttributeError:
    print('Max. time observable is already converted to minutes.')

In [9]:
visible_target

,Designation,Full name,Rise time,Transit time,Set time,Max. time observable,R.A.,Dec.,Vmag,Helio. range (au),Topo.range (au),Object-Observer-Sun (deg),Object-Observer-Moon (deg),Galactic latitude (deg),date_str,observatory,sb_class,fVmag
0,240P,240P/NEAT,11:23,14:42,18:01*,352,60.100,3.549,15.4N,2.15,1.19,159.94,49.83,-35.41,2025-11-10,lsgt,JFc,15.4
1,5D,5D/Brorsen,02:45*,07:47*,12:50,165,316.136,-37.740,16.6T,3.58,3.61,80.59,152.80,-41.39,2025-11-10,lsgt,JFc,16.6
2,48P,48P/Johnson,11:06,14:26,17:47*,369,56.184,3.193,17.0N,2.75,1.79,162.31,53.28,-38.71,2025-11-10,lsgt,JFc,17.0
3,47P,47P/Ashbrook-Jackson,05:32*,09:27*,13:21,196,341.015,-7.646,17.1T,2.82,2.29,112.17,124.20,-53.90,2025-11-10,lsgt,JFc,17.1
4,78P,78P/Gehrels 2,02:58*,07:08*,11:18,73,306.270,-13.524,17.4N,2.67,2.69,77.95,156.10,-26.49,2025-11-10,lsgt,JFc,17.4
5,199P,199P/Shoemaker 4,09:34*,13:33,17:33*,430,42.915,-9.445,17.5N,5.44,4.54,153.45,71.30,-56.91,2025-11-10,lsgt,JFc,17.5
6,332P,332P/Ikeya-Murakami,08:37*,11:31,14:24,259,12.188,9.806,17.6N,3.51,2.64,147.37,88.56,-53.20,2025-11-10,lsgt,ETc,17.6
7,2024 T5,C/2024 T5 (ATLAS),08:24*,13:43,19:02*,430,45.224,-45.947,16.4N,5.95,5.44,117.02,91.98,-58.36,2025-11-10,lsgt,HYP,16.4
8,2020 V2,C/2020 V2 (ZTF),23:36*,05:44*,11:52,107,285.319,-67.445,17.7T,8.70,9.10,63.15,140.50,-25.59,2025-11-10,lsgt,HYP,17.7
9,2022 R6,C/2022 R6 (PANSTARRS),15:32,19:39*,23:45*,102,134.466,-12.024,17.9N,6.59,6.56,86.96,48.66,20.98,2025-11-10,lsgt,HYP,17.9


In [63]:
pdes = "240P"

row = visible_target[visible_target.Designation == pdes].iloc[0]
epoch_start = pd.to_datetime(f"{row.date_str} {row['Rise time'].strip('*')}")
epoch_end = pd.to_datetime(f"{row.date_str} {row['Set time'].strip('*')}")

if epoch_end < epoch_start:
    epoch_end += pd.Timedelta(days=1)

epoch_start = epoch_start.strftime('%Y-%m-%d %H:%M')
epoch_end = epoch_end.strftime('%Y-%m-%d %H:%M')

epochs = {"start": epoch_start, "stop": epoch_end, "step": "30m"}
epochs

{'start': '2025-11-10 11:23', 'stop': '2025-11-10 18:01', 'step': '30m'}

In [64]:
# Query ephemeris
eph = sxobsplan.fetch_with_fallback(
    designation=pdes,
    epochs=epochs,
    quantities="1,3,4,9",
    location=dict_observatory[obsname][0]
)

eph = eph.to_pandas()
eph

2025-11-10 20:38:07,366 [INFO] Non-unique id for '240P'. Retrying with record #90001203 ...


,targetname,datetime_str,datetime_jd,M1,solar_presence,k1,lunar_presence,RA,DEC,RA_rate,DEC_rate,AZ,EL,Tmag,Nmag
0,240P-B/NEAT,2025-Nov-10 11:23,2.460990e+06,7.4,,30.25,,59.94026,3.83140,-36.6773,26.10310,61.016406,30.287038,17.792,<NA>
1,240P-B/NEAT,2025-Nov-10 11:53,2.460990e+06,7.4,,30.25,,59.93513,3.83502,-36.8438,26.10563,54.672028,35.585508,17.791,<NA>
2,240P-B/NEAT,2025-Nov-10 12:23,2.460990e+06,7.4,,30.25,,59.92998,3.83863,-36.9896,26.10686,47.407711,40.453063,17.791,<NA>
3,240P-B/NEAT,2025-Nov-10 12:53,2.460990e+06,7.4,,30.25,,59.92482,3.84225,-37.1121,26.10698,39.024852,44.740893,17.791,<NA>
4,240P-B/NEAT,2025-Nov-10 13:23,2.460990e+06,7.4,,30.25,,59.91964,3.84586,-37.2097,26.10621,29.378690,48.262156,17.791,<NA>
5,240P-B/NEAT,2025-Nov-10 13:53,2.460990e+06,7.4,,30.25,m,59.91444,3.84948,-37.2808,26.10476,18.479122,50.803769,17.790,<NA>
6,240P-B/NEAT,2025-Nov-10 14:23,2.460990e+06,7.4,,30.25,m,59.90924,3.85309,-37.3246,26.10289,6.607435,52.163800,17.790,<NA>
7,240P-B/NEAT,2025-Nov-10 14:53,2.460990e+06,7.4,,30.25,t,59.90404,3.85671,-37.3403,26.10084,354.346513,52.212443,17.790,<NA>
8,240P-B/NEAT,2025-Nov-10 15:23,2.460990e+06,7.4,,30.25,m,59.89883,3.86032,-37.3282,26.09887,342.423810,50.944688,17.789,<NA>
9,240P-B/NEAT,2025-Nov-10 15:53,2.460990e+06,7.4,,30.25,m,59.89363,3.86393,-37.2886,26.09721,331.440484,48.482553,17.789,<NA>


In [65]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.vizier import Vizier
from matplotlib.patches import Rectangle
from pathlib import Path
import pandas as pd
import sys

def plot_daily_finder_chart(eph, target_label, telescope_fov=(0.2, 0.2), output_dir=".", limit_mag=16.0):
    """
    Generates and saves a finder chart including the target's movement over a day.

    Args:
        eph (pd.DataFrame): Daily ephemeris data containing 'ra', 'dec' columns (in degrees).
                            If a 'datetime_str' column exists, it is used for labeling.
        target_label (str): Target name used for the chart title and filename.
        telescope_fov (tuple or float): Telescope FOV (width, height) [unit: degree].
                                        If a single float is provided, it is treated as a square.
        output_dir (str or Path): Directory path to save the chart.
        limit_mag (float): Limiting magnitude of stars to display (based on Gmag).
    """
    # --- 0. Setup and Preparation ---
    save_dir = Path(output_dir)
    save_dir.mkdir(parents=True, exist_ok=True)

    # If telescope FOV is a single value, set it as a square
    if isinstance(telescope_fov, (int, float)):
        tel_w, tel_h = telescope_fov, telescope_fov
    else:
        tel_w, tel_h = telescope_fov

    # Unify column names to lowercase for processing
    eph = eph.copy()
    eph.columns = [c.lower() for c in eph.columns]

    # --- 1. Calculate Chart Center and Size (FOV) ---
    ra_track = eph['ra'].values
    dec_track = eph['dec'].values

    # Calculate center coordinates (using mean values)
    center_ra = np.mean(ra_track)
    center_dec = np.mean(dec_track)
    center_dec_rad = np.deg2rad(center_dec)

    # Calculate projected track offsets
    # RA offset is multiplied by cos(dec) to correct for actual angular distance in the sky
    ra_offsets = (ra_track - center_ra) * np.cos(center_dec_rad)
    dec_offsets = (dec_track - center_dec)
    
    # Actual width and height occupied by the track
    track_width = np.max(ra_offsets) - np.min(ra_offsets)
    track_height = np.max(dec_offsets) - np.min(dec_offsets)

    # Set overall chart FOV: include both track size and telescope FOV, with 20% margin
    chart_fov = max(track_width + tel_w, track_height + tel_h) * 1.2
    # Ensure minimum FOV (in case movement is very small)
    chart_fov = max(chart_fov, max(tel_w, tel_h) * 2.0)

    # --- 2. Query Gaia DR3 Catalog ---
    center_coord = SkyCoord(ra=center_ra*u.deg, dec=center_dec*u.deg, frame='icrs')
    query_radius = (chart_fov / np.sqrt(2)) * u.deg
    
    print(f"[{target_label}] Creating chart... FOV: {chart_fov:.2f} deg")

    try:
        v = Vizier(columns=['RA_ICRS', 'DE_ICRS', 'Gmag'], catalog="I/355/gaiadr3")
        v.ROW_LIMIT = -1
        result = v.query_region(center_coord, radius=query_radius)
        if result and len(result) > 0:
            stars = result[0].to_pandas()
            # Filter stars that have magnitude data and are brighter than the limiting magnitude
            stars = stars[(~stars['Gmag'].isna()) & (stars['Gmag'] < limit_mag)]
        else:
            stars = pd.DataFrame()
    except Exception as e:
        print(f"Warning: Vizier query failed ({e})", file=sys.stderr)
        stars = pd.DataFrame()

    # --- 3. Draw Chart ---
    fig, ax = plt.subplots(figsize=(10, 10))

    # 3-1. Draw Background Stars
    if not stars.empty:
        star_x = (stars['RA_ICRS'].values - center_ra) * np.cos(center_dec_rad)
        star_y = stars['DE_ICRS'].values - center_dec
        # Draw brighter stars (lower magnitude) as larger points
        sizes = (limit_mag - stars['Gmag'].values)**1.8 + 2
        ax.scatter(star_x, star_y, s=sizes, c='k', marker='.', alpha=0.6, label='Stars', zorder=1)

    # 3-2. Draw Target Track (red dashed line)
    ax.plot(ra_offsets, dec_offsets, 'r--', lw=2, label='Track', zorder=2)

    # 3-3. Draw Telescope FOV Rectangles (Start, Mid, End)
    time_indices = [0, len(eph)//2, len(eph)-1] # Start, Mid, End indices
    
    for i, idx in enumerate(time_indices):
        # Center coordinates at that time
        cx, cy = ra_offsets[idx], dec_offsets[idx]
        # Bottom-left coordinates of the rectangle (based on matplotlib Rectangle)
        rect_x = cx - tel_w / 2
        rect_y = cy - tel_h / 2
        
        # Add red rectangle
        rect = Rectangle((rect_x, rect_y), tel_w, tel_h, 
                         linewidth=1.5, edgecolor='r', facecolor='none', ls='-', zorder=3)
        ax.add_patch(rect)

        # Time labeling (if datetime_str column exists)
        if 'datetime_str' in eph.columns:
            # Extract HH:MM from "YYYY-MM-DD HH:MM" format
            t_label = str(eph.iloc[idx]['datetime_str']).split(' ')[-1][:5]
            ax.text(cx, cy + tel_h/2 + chart_fov*0.01, t_label, 
                    color='r', ha='center', va='bottom', fontsize=9, weight='bold', zorder=4)

    # --- 4. Formatting and Saving ---
    # Invert RA axis (Astronomical convention: East is left)
    ax.set_xlim(chart_fov/2, -chart_fov/2)
    ax.set_ylim(-chart_fov/2, chart_fov/2)
    ax.set_aspect('equal')
    ax.grid(True, linestyle=':', alpha=0.5)
    ax.set_xlabel('RA Offset (deg) [East is Left]')
    ax.set_ylabel('Dec Offset (deg) [North is Up]')

    # Set title
    date_str = eph.iloc[0]['datetime_str'].split(' ')[0] if 'datetime_str' in eph.columns else ""
    ax.set_title(f"Finder Chart for {target_label} ({date_str})\nCenter RA: {center_ra:.4f}, Dec: {center_dec:.4f}")
    ax.legend(loc='upper right')

    # Direction indicators (N/E arrows) - Displayed in bottom left corner of chart
    arrow_len = chart_fov * 0.08
    base_x = -chart_fov * 0.42   # Positive X (Left side because of inverted axis)
    base_y = -chart_fov * 0.42  # Negative Y (Bottom side)

    # North Arrow
    ax.arrow(base_x, base_y, 0, arrow_len, head_width=arrow_len*0.25, fc='k', ec='k', zorder=5)
    ax.text(base_x, base_y + arrow_len*1.3, 'N', ha='center', va='bottom', fontweight='bold')
    # East Arrow (X-axis is inverted, so drawing in positive direction points left (East))
    ax.arrow(base_x, base_y, arrow_len, 0, head_width=arrow_len*0.25, fc='k', ec='k', zorder=5)
    ax.text(base_x + arrow_len*1.3, base_y, 'E', ha='right', va='center', fontweight='bold')

    plt.tight_layout()

    # Save to file
    # Include target name and date in filename
    safe_name = "".join(x for x in target_label if x.isalnum() or x in ['_', '-']).replace(' ', '_')
    filename = save_dir / f"finder_{safe_name}_{date_str}.png"
    plt.savefig(filename, dpi=150)
    print(f"Saved chart to: {filename}")
    plt.close(fig) # Close figure to free memory

In [66]:
plot_daily_finder_chart(eph,
                        pdes,
                        telescope_fov=dict_observatory[obsname][4].to_value("deg"),
                        output_dir=".",
                        limit_mag=18.0)

[240P] Creating chart... FOV: 0.52 deg
Saved chart to: finder_240P_2025-Nov-10.png
